**Homework 8**

We begin with the imports you will need, together with the functions defined in the last assignment. Note that a few have been added/modified! The Scaler class now scales each column of a feature matrix individually. There have also been utility functions added to add a column of 1's to a feature matrix, and compute the MSE.

Note that you won't use all functions below in all situations. For example, when doing polynomial regression, you won't need the AddOnes function. When doing linear regression with multiple features, you will.

In [1]:
import numpy as np
import pandas as pd

def TrainTestSplit(X,y,p,seed=1):
  '''Splits feature matrix X and target array y into train and test sets
  p is the fraction going to train'''
  np.random.seed(seed) #controls randomness
  size=len(y)
  train_size=int(p*size)
  train_mask=np.zeros(size,dtype=bool)
  train_indices=np.random.choice(size, train_size, replace=False)
  train_mask[train_indices]=True
  test_mask=~train_mask
  X_train=X[train_mask]
  X_test=X[test_mask]
  y_train=y[train_mask]
  y_test=y[test_mask]
  return X_train,X_test,y_train,y_test

def PolyFeatures(x,d):
  X=np.zeros((len(x),d+1))
  for i in range(d+1):
    X[:,i]=x**i
  return X

def AddOnes(X):
  return np.concatenate((X,np.ones((len(X),1))),axis=1)

class Scaler:
  '''scales columns of array individually'''
  def __init__(self,z):
    self.min=np.min(z,axis=0)
    self.max=np.max(z,axis=0)

  def scale(self,x):
    return (x-self.min)/(self.max-self.min)

  def unscale(self,x):
    return x*(self.max-self.min)+self.min

def train(X,y,max_iter,lr):
  '''MSE minimization by Gradient Descent'''
  X=np.array(X) #Just in case X is a DataFrame
  y=np.array(y) #Just in case y is a Series
  n=len(X)
  coeff=np.ones(X.shape[1]) #Initialize all coeff to be 1 (something to play with?)
  for i in range(max_iter):
    resid=X@coeff-y
    gradient=((X.T)@resid)/n #Lot's of lin alg here. Try to unpack it!
    coeff=coeff-lr*gradient #Gradient Descent step.
  return coeff

def predict(X,coeff): #If X was scaled, then this will return scaled predictions
  return X@coeff

def MSE(pred,y):
  return np.sum((pred-y)**2)/len(y)

In this assignment you will modify the `train` function above to work by Stochastic Gradient Descent. As discussed in class, SGD is more memory efficient and can lead to much faster convergence.

Be aware that if the batch size does not divide evenly into the size of your dataset, then the last batch will be smaller. Make sure your code doesn't leave out the last batch!

In [6]:
def SGD(X,y,epochs,batch_size,lr):
  '''Stochastic Gradient Descent'''
  X=np.array(X) #Just in case X is a DataFrame
  y=np.array(y) #Just in case y is a Series
  n=len(X)
  coeff=np.ones(X.shape[1]) #Initialize all coeff to be 1 (something to play with?)
  indices=np.arange(len(X))
  for i in range(epochs):
    np.random.seed(i) #Just so everyone gets the same answer!
    np.random.shuffle(indices)
    
    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        batch_indices = indices[start:end]
        X_batch = X[batch_indices]
        y_batch = y[batch_indices]
        
        # # Calculate the gradient for this batch
        # gradient = np.zeros(X.shape[1])
        # for j in range(len(batch_indices)):
        #     x_i = X_batch[j]
        #     y_i = y_batch[j]
        #     error = np.dot(coeff, x_i) - y_i
        #     gradient += 2 * x_i * error
            
        # # Update coefficients using the learning rate and the gradient
        # coeff -= lr * gradient / len(batch_indices)
        
        resid = X_batch@coeff-y_batch
        gradient = ((X_batch.T)@resid) / (end - start + 1) #Lot's of lin alg here. Try to unpack it!
        coeff = coeff - lr*gradient #Gradient Descent step.        
    
  return coeff

To test your code, we'll define 100,000 points around the line y=3x. The coefficient generated by linear regression should thus be about 3.

In [7]:
x=np.arange(0,1,.00001) #100,000 numbers between 0 and 1
np.random.seed(10) #Makes sure we all get the same answers!
random_nums=.4*np.random.rand(100000)
y=3*x+random_nums #Equation of y=3x, with some randomness
X=x.reshape((len(x),1)) #Necessary so X is a 2D feature matrix, rather than a 1D array.

Here's our original `train` function:

In [8]:
coeff=train(X,y,1000,.01)
coeff

array([3.21791934])

Now let's check your code, with 500 epochs and batches of size 1000. Recall that this means the computer has to keep only 1000 numbers in memory for computation at any given time, rather than all 100000.

In [9]:
coeff=SGD(X,y,500,1000,.01)
coeff

array([3.29945078])